In [ ]:
%matplotlib inline


Diferentes clasificadores en la decodificación del conjunto de datos Haxby
=====================================================

Comparación de diferentes clasificadores en una tarea de decodificación de reconocimiento de objetos visuales.

https://github.com/nilearn/nilearn/tree/master/examples



1) Cargar los datos y aplicarle transformaciones simples

In [ ]:
# Obtener datos usando la captura de conjunto de datos nilearn
from nilearn import datasets
haxby_dataset = datasets.fetch_haxby()

# imprimir información básica sobre el conjunto de datos
print('First subject anatomical nifti image (3D) located is at: %s' %
      haxby_dataset.anat[0])
print('First subject functional nifti image (4D) is located at: %s' %
      haxby_dataset.func[0])

# cargar etiquetas 
import numpy as np
import pandas as pd
labels = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
stimuli = labels['labels']
# identificar las etiquetas de estado en reposo para poder descartarlas
task_mask = (stimuli != 'rest')

# encontrar nombres de las etiquetas activas restantes
categories = stimuli[task_mask].unique()

# extraer etiquetas que indiquen a qué adquisición (estimulación) pertenece
session_labels = labels['chunks'][task_mask]

 # Cargar los datos de fMRI
from nilearn.input_data import NiftiMasker

# Para la decodificación, la estandarización a menudo es muy importante
mask_filename = haxby_dataset.mask_vt[0]
masker = NiftiMasker(mask_img=mask_filename, standardize=True)
func_filename = haxby_dataset.func[0]
masked_timecourses = masker.fit_transform(func_filename)[task_mask]

2) Definir los diferentes clasificadores a usar


In [ ]:
# Clasificador de vectores de soporte
from sklearn.svm import SVC
svm = SVC(C=1., kernel="linear")

# Regresión logistica
from sklearn.linear_model import (LogisticRegression,
                                  RidgeClassifier,
                                  RidgeClassifierCV,
                                  )
logistic = LogisticRegression(C=1., penalty="l1")
logistic_50 = LogisticRegression(C=50., penalty="l1")
logistic_l2 = LogisticRegression(C=1., penalty="l2")

# Validación cruzada de clasificadores
from sklearn.model_selection import GridSearchCV

svm_cv = GridSearchCV(SVC(C=1., kernel="linear"),
                      param_grid={'C': [.1, .5, 1., 5., 10., 50., 100.]},
                      scoring='f1', n_jobs=1)

logistic_cv = GridSearchCV(LogisticRegression(C=1., penalty="l1"),
                           param_grid={'C': [.1, .5, 1., 5., 10., 50., 100.]},
                           scoring='f1')
logistic_l2_cv = GridSearchCV(LogisticRegression(C=1., penalty="l2"),
                              param_grid={
                                  'C': [.1, .5, 1., 5., 10., 50., 100.]
                              },
                              scoring='f1')


# El clasificador de cresta tiene un objeto 'CV' que puede establecer sus
# parámetros más rápidos que usar un GridSearchCV
ridge = RidgeClassifier()
ridge_cv = RidgeClassifierCV()

# Diccionario que contiene todos nuestros clasificadores
classifiers = {'SVC': svm,
               'SVC cv': svm_cv,
               'log l1': logistic,
               'log l1 50': logistic_50,
               'log l1 cv': logistic_cv,
               'log l2': logistic_l2,
               'log l2 cv': logistic_l2_cv,
               'ridge': ridge,
               'ridge cv': ridge_cv
               }

3) Calcular puntajes de predicción

In [ ]:
# Objeto de división de datos para la validación cruzada
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
cv = LeaveOneGroupOut()

import time

classifiers_scores = {}

for classifier_name, classifier in sorted(classifiers.items()):
    classifiers_scores[classifier_name] = {}
    print(70 * '_')

    for category in categories:
        classification_target = stimuli[task_mask].isin([category])
        t0 = time.time()
        classifiers_scores[classifier_name][category] = cross_val_score(
            classifier,
            masked_timecourses,
            classification_target,
            cv=cv,
            groups=session_labels,
            scoring="f1",
        )

        print(
            "%10s: %14s -- scores: %1.2f +- %1.2f, time %.2fs" %
            (
                classifier_name,
                category,
                classifiers_scores[classifier_name][category].mean(),
                classifiers_scores[classifier_name][category].std(),
                time.time() - t0,
            ),
        )

3) Graficar diagrama



In [ ]:
import matplotlib.pyplot as plt
plt.figure()

tick_position = np.arange(len(categories))
plt.xticks(tick_position, categories, rotation=45)

for color, classifier_name in zip(
        ['b', 'c', 'm', 'g', 'y', 'k', '.5', 'r', '#ffaaaa'],
        sorted(classifiers)):
    score_means = [classifiers_scores[classifier_name][category].mean()
                   for category in categories]
    plt.bar(tick_position, score_means, label=classifier_name,
            width=.11, color=color)
    tick_position = tick_position + .09

plt.ylabel('Classification accurancy (f1 score)')
plt.xlabel('Visual stimuli category')
plt.ylim(ymin=0)
plt.legend(loc='lower center', ncol=3)
plt.title(
    'Category-specific classification accuracy for different classifiers')
plt.tight_layout()

4)  Trazar el mapa de 'face' frente a 'house' para los diferentes clasificadores



In [ ]:
# Usa promedio fondo
from nilearn import image
mean_epi_img = image.mean_img(func_filename)

# Restringir la decodificación para enfrentar 'face' frente 'house'
condition_mask = stimuli.isin(['face', 'house'])
masked_timecourses = masked_timecourses[
    condition_mask[task_mask]]
stimuli = (stimuli[condition_mask] == 'face')
# Transformar los estímulos en valores binarios
stimuli.astype(np.int)

from nilearn.plotting import plot_stat_map, show

for classifier_name, classifier in sorted(classifiers.items()):
    classifier.fit(masked_timecourses, stimuli)

    if hasattr(classifier, 'coef_'):
        weights = classifier.coef_[0]
    elif hasattr(classifier, 'best_estimator_'):
        weights = classifier.best_estimator_.coef_[0]
    else:
        continue
    weight_img = masker.inverse_transform(weights)
    weight_map = weight_img.get_data()
    threshold = np.max(np.abs(weight_map)) * 1e-3
    plot_stat_map(weight_img, bg_img=mean_epi_img,
                  display_mode='z', cut_coords=[-15],
                  threshold=threshold,
                  title='%s: face vs house' % classifier_name)

show()